# Multi-swarm Test

In [ ]:
# Agent config
num_agents_1 = 100
num_agents_2 = 100
agent_radius = 1 

# Map config
map_type = "cross" # Available maps: empty, corridor, obstacle, cross
num_samples = 100    # Available num_samples: 100, 200, 400, 500

# Instance config, ratio of starts and goals

starts_weight_1 = [1, 0]
goals_weight_1 = [1, 0]
starts_weight_2 = [0, 1]
goals_weight_2 = [0, 1]


In [ ]:
# Load map and instance
import os
import pickle

from swarm_prm.utils.agent_assignment import get_agent_assignment

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_count_1 = get_agent_assignment(num_agents_1, starts_weight_1)
goals_agent_count_1 = get_agent_assignment(num_agents_1, goals_weight_1)

starts_agent_count_2 = get_agent_assignment(num_agents_2, starts_weight_2)
goals_agent_count_2 = get_agent_assignment(num_agents_2, goals_weight_2)

print("team 1")
print(starts_agent_count_1, goals_agent_count_1)
print("team 2")
print(starts_agent_count_2, goals_agent_count_2)

print(gaussian_prm.starts_idx, gaussian_prm.goals_idx)


In [ ]:
# TEG
from swarm_prm.solvers.macro.teg import TEG

# Plan for first swarm
teg = TEG(gaussian_prm, agent_radius, 
              starts_agent_count=starts_agent_count_1, goals_agent_count=goals_agent_count_1,
               num_agents=num_agents_1, time_limit=10)
timestep, flow_dict_1, capacity_dict= teg.get_solution()

# Plan for second swram
teg = TEG(gaussian_prm, agent_radius, 
              starts_agent_count=starts_agent_count_2, goals_agent_count=goals_agent_count_2,
               num_agents=num_agents_2, 
               timestep=timestep,
               flow_dict=flow_dict_1,
               capacity_dict=capacity_dict,
               time_limit=10)
timestep, flow_dict_2, _ = teg.get_solution() 


In [ ]:
from matplotlib import pyplot as plt
from swarm_prm.solvers.micro import GaussianTrajectorySolver

simple_paths, _ = gaussian_prm.get_solution(flow_dict_1, timestep, num_agents_1)
macro_solution = gaussian_prm.get_macro_solution(flow_dict_1)

timestep = max(macro_solution.keys())

gt_solver = GaussianTrajectorySolver(gaussian_prm, macro_solution, timestep, num_agents_1, 
                                     starts_agent_count=starts_agent_count_1,
                                     goals_agent_count=goals_agent_count_1,
                                     safety_gap=0)

gt_paths_1 = gt_solver.solve()

simple_paths, _ = gaussian_prm.get_solution(flow_dict_2, timestep, num_agents_1)
macro_solution = gaussian_prm.get_macro_solution(flow_dict_2)

timestep = max(macro_solution.keys())

gt_solver = GaussianTrajectorySolver(gaussian_prm, macro_solution, timestep, num_agents_2, 
                                     starts_agent_count=starts_agent_count_2,
                                     goals_agent_count=goals_agent_count_2,
                                     safety_gap=0)

gt_paths_2 = gt_solver.solve()


fig, ax = gaussian_prm.visualize_g_nodes()

for path in simple_paths:
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
# 
cmap = plt.get_cmap("tab10")
# 
for i, path in enumerate(gt_paths_1):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '--', label='Path', color="red")

for i, path in enumerate(gt_paths_2):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '--', label='Path', color="green")

plt.show()

In [ ]:
# Add interpolations 

import numpy as np

def add_linear_interpolation_points(path, subdiv=5):
    """
    Insert linearly-interpolated positions between consecutive points.
    
    Parameters
    ----------
    path : array-like of shape (N, 2)
        The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
    subdiv : int
        Number of intervals to subdivide each segment. 
        For each original segment, we will add 'subdiv - 1' new interior points 
        (except that we skip duplicates at the shared boundary).

    Returns
    -------
    new_path : np.ndarray of shape (M, 2)
        The new path including the original points and the added interpolation points.
    """
    path = np.asarray(path)
    new_path = []

    # Iterate over pairs of consecutive points
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        
        # Create subdiv+1 points in [start, end] using linspace for each dimension
        xs = np.linspace(start[0], end[0], subdiv + 1)
        ys = np.linspace(start[1], end[1], subdiv + 1)
        
        # Add each intermediate point except the very last one
        # to avoid duplicating the next segment's start
        for j in range(subdiv):
            new_path.append([xs[j], ys[j]])

    # Finally add the very last point of the last segment
    new_path.append(path[-1].tolist())
    
    return np.array(new_path)

interpolated_paths = []
for path in gt_paths_1:
    new_path = add_linear_interpolation_points(path, 50)
    interpolated_paths.append(new_path)

for path in gt_paths_2:
    new_path = add_linear_interpolation_points(path, 50)
    interpolated_paths.append(new_path)


In [ ]:
# Animate solution
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

def animate_solution(fig, ax, speed, paths):
    """
        Visualize solution trajectory provided instance
    """
    
    agents = []
    def init():
        for agent in agents:
            agent.remove()
        agents.clear()
        return []

    def update(frame):
        idx = frame * speed
        for agent in agents:
            agent.remove()
        agents.clear()
        cmap = plt.get_cmap("tab10")
        locs = [path[idx] for path in paths]
        for i, loc in enumerate(locs):
            agent = ax.add_patch(Circle(loc, radius=agent_radius, color=cmap(i%10)))
            agents.append(agent)
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0])// speed , 
                         init_func=init, blit=True, interval=10)
    anim.save("solutions/apf_solution.gif", writer='pillow', fps=6)

# fig, ax = instance.visualize()
fig, ax = gaussian_prm.visualize_g_nodes()
animate_solution(fig, ax, 10, interpolated_paths)